In [ ]:
from commonfunctions import *
import math
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
img = read_img("/home/kamel/Desktop/Image/Project/kamel_trial/test.png")
bin_img = binarize_img (rgb2gray(img))
show_images([img,bin_img],["Original","Binarized"])

In [5]:
import numpy as np
import cv2
import math
from scipy import ndimage



def rotate_img(img,img_edges):
    img_copy = np.copy(img)
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
    angles = []
    for [[x1, y1, x2, y2]] in lines:
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
        angles.append(angle)
    median_angle = np.median(angles)
    img_rotated = ndimage.rotate(img_copy, median_angle)
    cv2.imwrite('fixed.jpg', img_rotated)  

img = cv2.imread('/home/kamel/Desktop/Image/Project/OMR_IP/input/test.png')
img = ndimage.rotate(img, -20)
cv2.imwrite('Original.jpg', img)  
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
cv2.imwrite('edges.jpg', img_edges)  
rotate_img(img,img_edges)